<a href="https://colab.research.google.com/github/machiwao/CCMACLRL_PROJECT_COM222/blob/main/predicting_bank_customer_monthly_incomes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Bank Customer's Monthly Income
Dataset provided by BPI for their Hackathon entitled "BPI DATA Wave 2024 : Data-Driven and AI-powered Solutions for MSMEs: Fueling Growth with Innovation." It contains Bank Customer Profile with personally identifiable information (PII) removed.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
CDNA = 'https://github.com/machiwao/bpi_msme_financial_lifeline/raw/refs/heads/main/dataset/CDNA_Q42023_Q12024.parquet'

## Data Loading

In [3]:
cdna_df = pd.read_parquet(CDNA)

In [5]:
cdna_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92204 entries, 0 to 92203
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   CUSTOMER_ID                  92204 non-null  float64
 1   GENDER                       92204 non-null  object 
 2   AGE                          89725 non-null  float64
 3   MARITAL_STATUS               92204 non-null  object 
 4   EDUCATION                    92204 non-null  object 
 5   BANK_TENURE                  89928 non-null  float64
 6   CUSTOMER_LOCATION            92204 non-null  object 
 7   DIGITAL_INDICATOR            92204 non-null  object 
 8   SEC                          92204 non-null  object 
 9   HOME_OWNER_INDICATOR         92204 non-null  object 
 10  CAR_OWNER_INDICATOR          92204 non-null  object 
 11  MONTHLY_INCOME               91309 non-null  float64
 12  INCOME_SOURCE                92204 non-null  object 
 13  ENVIRONMENTAL_AF

Feature Descriptions:

CUSTOMER_ID - Client ID of customer

GENDER - Gender of customer

AGE - Age of customer

MARITAL_STATUS Marital Status of customer

EDUCATION - Level of Educational Attainment

BANK_TENURE - Client Tenure in years with the bank (continuous)

CUSTOMER_LOCATION - Client location (Region)

DIGITAL_INDICATOR - Classifies the client whether they prefer traditional or digital channels

SEC - SocioEconomic Class

HOME_OWNER_INDICATOR - Home owner indicator whether customer owns a home or not

CAR_OWNER_INDICATOR - Car owner indicator whether customer owns a car or not

MONTHLY_INCOME Customer's Monthly Income (target)

INCOME_SOURCE Customer declared source of income

ENVIRONMENTAL_AFF_INDICATOR - Customer indicator whether transacted with an environmental group

HUMANITARIAN_AFF_INDICATOR - Customer indicator whether transacted with an humanitarian group

OF_INDICATOR - Customer indicator for Overseas Filipino clients

RETIREES_INDICATOR - Customer indicator for Retiree clients

FILCHI_INDICATOR - Customer indicator for Filipino-Chinese clients

In [9]:
cdna_df.head()

,CUSTOMER_ID,GENDER,AGE,MARITAL_STATUS,EDUCATION,BANK_TENURE,CUSTOMER_LOCATION,DIGITAL_INDICATOR,SEC,HOME_OWNER_INDICATOR,CAR_OWNER_INDICATOR,MONTHLY_INCOME,INCOME_SOURCE,ENVIRONMENTAL_AFF_INDICATOR,HUMANITARIAN_AFF_INDICATOR,OF_INDICATOR,RETIREES_INDICATOR,FILCHI_INDICATOR
0,4.598716e+06,FEMALE,43.67,MARRIED,MID,15.50,REGION VII (CENTRAL VISAYAS),TRADITIONAL,C1,Y,N,59465.23,COMMISSION,N,N,N,N,N
1,7.509760e+06,FEMALE,29.74,SINGLE,NO_DATA,2.59,NATIONAL CAPITAL REGION,DIGITAL,C1,Y,N,74533.25,SALARY,N,N,N,N,N
2,9.597760e+06,MALE,30.80,SINGLE,NO_DATA,1.92,NATIONAL CAPITAL REGION,DIGITAL,B2,Y,N,102944.40,SALARY,N,N,N,N,N
3,1.749770e+07,MALE,31.85,SINGLE,MID,14.47,NATIONAL CAPITAL REGION,DIGITAL,B2,Y,N,115609.09,SALARY,N,N,Y,N,N
4,1.758271e+07,FEMALE,53.92,SINGLE,MID,27.25,NATIONAL CAPITAL REGION,DIGITAL,B2,Y,N,128745.85,REMITTANCE,N,N,Y,N,N


In [15]:
for label in cdna_df.columns:
    print(cdna_df[label].unique())
    print()

[4.59871571e+06 7.50976000e+06 9.59776000e+06 ... 1.41429000e+08
 1.41884443e+08 1.12358171e+07]

['FEMALE' 'MALE' 'NO_DATA']

[43.67 29.74 30.8  ... 14.74 97.17 17.72]

['MARRIED' 'SINGLE' 'NO_DATA' 'SEPARATED' 'OTHER' 'WIDOW(ER)' 'DIVORCED']

['MID' 'NO_DATA' 'HIGH' 'LOW']

[15.5   2.59  1.92 ... 30.61 29.76 28.12]

['REGION VII (CENTRAL VISAYAS)' 'NATIONAL CAPITAL REGION'
 'REGION VI (WESTERN VISAYAS)' 'REGION III (CENTRAL LUZON)'
 'REGION I (ILOCOS REGION)' 'NO_DATA' 'REGION IV-A (CALABARZON)'
 'REGION V (BICOL REGION)' 'REGION XI (DAVAO REGION)'
 'CORDILLERA ADMINISTRATIVE REGION (CAR)'
 'REGION IX (ZAMBOANGA PENINSULA)' 'REGION XIII (CARAGA)'
 'REGION X (NORTHERN MINDANAO)' 'REGION XII (SOCCSKSARGEN)'
 'REGION II (CAGAYAN VALLEY)' 'REGION VIII (EASTERN VISAYAS)'
 'MIMAROPA REGION']

['TRADITIONAL' 'DIGITAL' 'NO_DATA']

['C1' 'B2' 'A' 'C2' 'B1' 'E' 'D' 'NO_DATA']

['Y' 'N']

['N' 'Y']

[5.94652300e+04 7.45332500e+04 1.02944400e+05 ... 2.37800000e+01
 9.50912100e+04 1.01708867e+06]

In [14]:
cdna_df['CUSTOMER_ID'].value_counts()

,count
CUSTOMER_ID,
4.598716e+06,1
9.532852e+07,1
9.532680e+07,1
9.532677e+07,1
9.532533e+07,1
...,...
4.742317e+07,1
4.742173e+07,1
4.742106e+07,1


In [6]:
cdna_df.describe()

,CUSTOMER_ID,AGE,BANK_TENURE,MONTHLY_INCOME
count,9.220400e+04,89725.000000,89928.000000,9.130900e+04
mean,7.132764e+07,41.607609,7.949568,1.004073e+05
std,4.129955e+07,15.590257,6.843168,1.088005e+06
min,9.457143e+02,0.170000,-3.430000,-4.000000e+00
25%,3.546922e+07,29.850000,2.300000,8.943800e+03
50%,7.124795e+07,37.940000,6.470000,1.816707e+04
75%,1.072326e+08,50.610000,11.720000,4.999982e+04
max,1.428542e+08,174.310000,33.320000,1.311747e+08


In [7]:
cdna_df.isna().sum()

,0
CUSTOMER_ID,0
GENDER,0
AGE,2479
MARITAL_STATUS,0
EDUCATION,0
BANK_TENURE,2276
CUSTOMER_LOCATION,0
DIGITAL_INDICATOR,0
SEC,0
HOME_OWNER_INDICATOR,0
